In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
directory = '../../cc3/rice_data/ecg_ppg_25_left4/'
yield_ecg = []
yield_ppg = []
for f in os.listdir(directory):
    if f[-1]!='p':
        continue
    data = pickle.load(open(directory+f,'rb'))[0]
    data_days = [df for i,df in data.groupby(['user','day'],as_index=False) if df.shape[0]>0]
    for df in data_days:
        if df['stress_likelihood_ppg'].dropna().shape[0]>0:
            yield_ppg.append(df['stress_likelihood_ppg'].dropna().shape[0])
        if df['stress_likelihood_ecg'].dropna().shape[0]>0:
            yield_ecg.append(df['stress_likelihood_ecg'].dropna().shape[0])

In [ ]:
plt.rcParams.update({'font.size':20})
plt.figure(figsize=(16,8))
plt.boxplot([yield_ppg,yield_ecg])
plt.xticks([1,2],['PPG yield. Days = '+str(len(yield_ppg)),'ECG yield. Days = '+str(len(yield_ecg))])
plt.ylabel('Minutes')
plt.show()

In [ ]:
from sklearn.metrics import r2_score
directory = '../../cc3/rice_data/ecg_ppg_25_left4/'
all_scores = []
for f in os.listdir(directory):
    if f[-1]!='p':
        continue
    data = pickle.load(open(directory+f,'rb'))[0]
    data_days = [df[['stress_likelihood_ecg',
                     'stress_likelihood_ppg',
                     'stress_likelihood_ppg_qual']].dropna() for i,df in data.groupby(['user','day'],as_index=False) if df.shape[0]>0]
    scores = [np.array([r2_score(df['stress_likelihood_ecg'],df['stress_likelihood_ppg']), \
                        r2_score(df['stress_likelihood_ecg'],df['stress_likelihood_ppg_qual'])]) for df in data_days if df.shape[0]>60]
    all_scores.extend(scores)

In [ ]:
all_scores = np.array(all_scores)
plt.figure(figsize=(16,8))
plt.boxplot(all_scores)
plt.xticks([1,2],['PPG Stress with weighted features,\n original Normalization',
                 'PPG Stress with weighted features,\n weighted Normalization'])
plt.ylabel('R squared value')
plt.ylim([-1,1])
plt.show()

In [ ]:
from sklearn.metrics import r2_score
import pickle
import numpy as np
directory = '../../cc3/rice_data/ecg_ppg_25_left4/'
all_scores_1 = []
all_scores_2 = []
all_yields = []
# data_col = []

# for f in os.listdir(directory):
#     if f[-1]!='p':
#         continue
#     data = pickle.load(open(directory+f,'rb'))[0]
#     data_col.append(data)
print(len(data_col))
for k in np.arange(0,1,.1):
    all_scores = []
    yields = []
    for data in data_col:
        data_days = [df[['stress_likelihood_ecg',
                         'stress_likelihood_ppg',
                         'stress_likelihood_ppg_qual',
                         'quality_mag']] for i,df in data.groupby(['user','day'],as_index=False) if df.shape[0]>0]
        yields.extend([df[df.quality_mag>=k]['stress_likelihood_ppg'].dropna().shape[0] for df in data_days])
        data_days = [a[a.quality_mag>=k].dropna() for a in data_days if a[a.quality_mag>=k].dropna().shape[0]>60]
        try:
            scores = [np.array([r2_score(df['stress_likelihood_ecg'],df['stress_likelihood_ppg']), \
                            r2_score(df['stress_likelihood_ecg'],df['stress_likelihood_ppg_qual'])]) for df in data_days]
        except:
            scores = [np.zeros(0,2)]
        all_scores.extend(scores)
    all_yields.append(np.array(yields))
    all_scores = np.array(all_scores)
    all_scores_1.append(list(all_scores[:,0]))
    all_scores_2.append(list(all_scores[:,1]))        

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':20})
fig,ax1 = plt.subplots(2,1,figsize=(20,10))
ax1[1].boxplot(all_yields, positions = np.array(range(0,10,1)), widths = 0.2)
ax1[1].set_xticklabels([str(np.round(k*100)/100) for k in np.arange(0,1,.1)])
ax1[1].set_xticks(range(0,10,1))
ax1[1].set_ylabel('Minutes')
ax = ax1[0]
bp = ax.boxplot(all_scores_1, positions = np.array(range(1,11,1)), widths = 0.2)
for i in range(10):
    plt.setp(bp['boxes'][i], color='blue')
# second boxplot pair
bp = ax.boxplot(all_scores_2, positions = np.array(range(1,11,1))+.25, widths = 0.2)
# setBoxColors(bp)
for i in range(10):
    plt.setp(bp['boxes'][i], color='red')
ax.set_ylim(-2,1)
ax.set_xticklabels([str(np.round(k*100)/100) for k in np.arange(0,1,.1)])
ax.set_xticks(range(1,11,1))
ax.set_ylabel('R Squared value')
# draw temporary red and blue lines and use them to create a legend
hB, = plt.plot([1,1],'b-')
hR, = plt.plot([1,1],'r-')
ax.legend((hB, hR),('Original Normalization', 'Weighted Normalization'))
hB.set_visible(False)
hR.set_visible(False)
plt.show()

In [ ]:
def KL(P,Q):
    """ Epsilon is used here to avoid conditional code for
    checking that neither P nor Q is equal to 0. """
    epsilon = 0.00001

     # You may want to instead make copies to avoid changing the np arrays.
    P = P+epsilon
    Q = Q+epsilon

    divergence = np.sum(P*np.log(P/Q))
    return divergence
from sklearn.metrics import r2_score
import pickle
import numpy as np
directory = '../../cc3/rice_data/ecg_ppg_25_left4/'
all_scores_1 = []
all_scores_2 = []
all_yields = []
for k in np.arange(0,1,.1):
    all_scores = []
    yields = []
    for data in data_col:
        data_days = [df[['stress_likelihood_ecg',
                         'stress_likelihood_ppg',
                         'stress_likelihood_ppg_qual',
                         'quality_mag']] for i,df in data.groupby(['user','day'],as_index=False) if df.shape[0]>0]
        yields.extend([df[df.quality_mag>=k]['stress_likelihood_ppg'].dropna().shape[0] for df in data_days])
        data_days = [a[a.quality_mag>=k].dropna() for a in data_days if a[a.quality_mag>=k].dropna().shape[0]>60]
        try:
            scores = [np.array([KL(df['stress_likelihood_ecg'],df['stress_likelihood_ppg']), \
                            KL(df['stress_likelihood_ecg'],df['stress_likelihood_ppg_qual'])]) for df in data_days]
        except:
            scores = [np.zeros(0,2)]
        all_scores.extend(scores)
    all_yields.append(np.array(yields))
    all_scores = np.array(all_scores)
    all_scores_1.append(list(all_scores[:,0]))
    all_scores_2.append(list(all_scores[:,1]))        

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':20})
fig,ax1 = plt.subplots(2,1,figsize=(20,10))
ax1[1].boxplot(all_yields, positions = np.array(range(0,10,1)), widths = 0.2)
ax1[1].set_xticklabels([str(np.round(k*100)/100) for k in np.arange(0,1,.1)])
ax1[1].set_xticks(range(0,10,1))
ax1[1].set_ylabel('Minutes')
ax = ax1[0]
bp = ax.boxplot(all_scores_1, positions = np.array(range(1,11,1)), widths = 0.2)
for i in range(10):
    plt.setp(bp['boxes'][i], color='blue')
bp = ax.boxplot(all_scores_2, positions = np.array(range(1,11,1))+.25, widths = 0.2)
for i in range(10):
    plt.setp(bp['boxes'][i], color='red')
ax.set_xticklabels([str(np.round(k*100)/100) for k in np.arange(0,1,.1)])
ax.set_xticks(range(1,11,1))
ax.set_ylabel('KL Divergence')
hB, = plt.plot([1,1],'b-')
hR, = plt.plot([1,1],'r-')
ax.legend((hB, hR),('Original Normalization', 'Weighted Normalization'))
hB.set_visible(False)
hR.set_visible(False)
plt.show()

In [ ]:
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
import pickle
import numpy as np
directory = '../../cc3/rice_data/ecg_ppg_25_left4/'
all_scores_1 = []
all_scores_2 = []
all_yields = []
for k in np.arange(0,1,.1):
    all_scores = []
    yields = []
    for data in data_col:
        data_days = [df[['stress_likelihood_ecg',
                         'stress_likelihood_ppg',
                         'stress_likelihood_ppg_qual',
                         'quality_mag']] for i,df in data.groupby(['user','day'],as_index=False) if df.shape[0]>0]
        yields.extend([df[df.quality_mag>=k]['stress_likelihood_ppg'].dropna().shape[0] for df in data_days])
        data_days = [a[a.quality_mag>=k].dropna() for a in data_days if a[a.quality_mag>=k].dropna().shape[0]>60]
        try:
            scores = [np.array([pearsonr(df['stress_likelihood_ecg'],df['stress_likelihood_ppg'])[0], \
                            pearsonr(df['stress_likelihood_ecg'],df['stress_likelihood_ppg_qual'])[0]]) for df in data_days]
        except:
            scores = [np.zeros(0,2)]
        all_scores.extend(scores)
    all_yields.append(np.array(yields))
    all_scores = np.array(all_scores)
    all_scores_1.append(list(all_scores[:,0]))
    all_scores_2.append(list(all_scores[:,1]))        

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':20})
fig,ax1 = plt.subplots(2,1,figsize=(20,10))
ax1[1].boxplot(all_yields, positions = np.array(range(0,10,1)), widths = 0.2)
ax1[1].set_xticklabels([str(np.round(k*100)/100) for k in np.arange(0,1,.1)])
ax1[1].set_xticks(range(0,10,1))
ax1[1].set_ylabel('Minutes')
ax = ax1[0]
bp = ax.boxplot(all_scores_1, positions = np.array(range(1,11,1)), widths = 0.2)
for i in range(10):
    plt.setp(bp['boxes'][i], color='blue')
bp = ax.boxplot(all_scores_2, positions = np.array(range(1,11,1))+.25, widths = 0.2)
for i in range(10):
    plt.setp(bp['boxes'][i], color='red')
ax.set_xticklabels([str(np.round(k*100)/100) for k in np.arange(0,1,.1)])
ax.set_xticks(range(1,11,1))
ax.set_ylabel('Pearson Correlation')
hB, = plt.plot([1,1],'b-')
hR, = plt.plot([1,1],'r-')
ax.legend((hB, hR),('Original Normalization', 'Weighted Normalization'))
hB.set_visible(False)
hR.set_visible(False)
plt.show()

In [ ]:
all_yields

In [10]:
data_col[0].head()

,window,ltime,likelihood_max_array,activity,rr_array,time,timestamp,likelihood_mean,localtime,ecg_rr_array,...,length1,indicator,mean,std,rr_features,rr_weighted_features,quality_mag,stress_likelihood_ppg_qual,stress_likelihood_ecg,stress_likelihood_ppg
0,"(2019-08-12 21:01:00, 2019-08-12 21:02:00)",2019-08-12 20:01:14.039562,"[[0.11416666666666667, 0.04583333333333333, 0....","[0.009696945451220504, 0.006266727846887542, 0...","[[0.0, 0.0, 637.9585326953747], [892.857142857...",1.565658e+09,2019-08-13 02:01:00.025266,0.524333,2019-08-12 20:01:00.025266,"[698.0, 744.0, 756.0, 726.0, 763.0, 660.0, 746...",...,47,Available,692.641474,135.208628,"[1.1301980630338906, 0.9968226662237158, -0.64...","[0.8830488406635035, 0.8338138816641115, -0.51...",0.493769,0.215102,0.140221,0.195026
1,"(2019-08-12 21:14:00, 2019-08-12 21:15:00)",2019-08-12 20:14:08.015627,"[[0.1075, 0.025, 0.8850000000000001], [0.58716...","[0.004843202000653242, 0.024417709811546166, 0...","[[877.1929824561403, 491.4004914004914, 704.22...",1.565659e+09,2019-08-13 02:14:00.025488,0.616000,2019-08-12 20:14:00.025488,"[805.0, 680.0, 838.0, 723.0, 691.0, 785.0, 813...",...,54,Available,792.116358,190.360331,"[2.24026115300184, 1.5384638157822477, 0.13500...","[1.2210400813035955, 1.265741803178839, 0.1291...",0.553965,0.047184,0.035346,0.047141
2,"(2019-08-12 21:18:00, 2019-08-12 21:19:00)",2019-08-12 20:18:45.005643,"[[0.43599999999999994, 0.4660000000000001, 0.6...","[0.03649806978521407, 0.005991802233032004, 0....","[[1123.5955056179776, 1010.1010101010102, 845....",1.565659e+09,2019-08-13 02:18:00.032417,0.576667,2019-08-12 20:18:00.032417,"[781.0, 791.0, 666.0, 797.0, 793.0, 875.0, 697...",...,54,Available,794.920430,210.230694,"[2.7323604881096433, 2.2343636392331945, 0.157...","[1.5886100760476836, 0.9479024704921017, 0.254...",0.515369,0.033983,0.054483,0.034423
3,"(2019-08-12 22:37:00, 2019-08-12 22:38:00)",2019-08-12 21:37:16.011222,"[[0.193, 0.22916666666666663, 1.0], [0.1933333...","[0.015765911601970765, 0.012975097062291035, 0...","[[1111.111111111111, 1055.4089709762532, 854.7...",1.565664e+09,2019-08-13 03:37:00.032202,0.863667,2019-08-12 21:37:00.032202,"[845.0, 979.0, 808.0, 806.0, 838.0, 688.0, 801...",...,58,Available,793.649757,68.274437,"[0.2881787806857096, 0.6565221663507733, 0.147...","[0.269848681291395, 0.6293138391352854, 0.1782...",0.866322,0.056494,0.036474,0.063015
4,"(2019-08-12 22:49:00, 2019-08-12 22:50:00)",2019-08-12 21:49:35.032527,"[[0.04, 0.04166666666666666, 0.134999999999999...","[0.04297184767324085, 0.01853200582809977, 0.0...","[[525.6241787122208, 509.55414012738845, 536.9...",1.565665e+09,2019-08-13 03:49:00.003167,0.667667,2019-08-12 21:49:00.003167,"[902.0, 797.0, 691.0, 822.0, 700.0, 778.0, 841...",...,48,Available,752.422467,137.688208,"[1.1720313852887188, 1.3135897677533777, -0.17...","[0.7995933097537749, 0.7574712485513633, -0.09...",0.576088,0.056162,0.029051,0.058188


In [2]:
from fireTS.models import NARX, DirectAutoRegressor

ModuleNotFoundError: No module named 'fireTS'